In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("../codes_dataset.xlsx", sheet_name="belgian_filtered")

In [6]:
df.columns

Index(['SurveyID', 'SurveySessionID', 'SessionStartTime', 'SessionEndTime',
       'SessionFeedback1', 'SessionFeedback2', 'ResponseId', 'QuestionID',
       'SubQuestionID', 'QuestionText', 'ResponseText', 'ResponseOptionId',
       'ClassificationAttempts', 'ClassificationOptionID',
       'ClassificationText', 'PercentageLikelihood', 'IsUserConfirmed',
       'SubclassificationOptionID', 'SubclassificationText', 'ActionPhrase',
       'SubclassificationPercentageLikelihood',
       'SubclassificationIsUserConfirmed', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'MainCategoryCorrect', 'AttemptID', 'Unnamed: 27',
       'UserConfirmedQ', 'UserInputCorrect', 'FailureReason',
       'SubclassPresent'],
      dtype='object')

In [9]:
df['IsUserConfirmed']

0       True
1      False
2      False
3      False
4       True
       ...  
197     True
198    False
199    False
200     True
201     True
Name: IsUserConfirmed, Length: 202, dtype: bool

In [5]:
df.describe()

,QuestionID,SubQuestionID,ResponseOptionId,ClassificationAttempts,ClassificationOptionID,PercentageLikelihood,SubclassificationOptionID,SubclassificationPercentageLikelihood,SubclassificationIsUserConfirmed,Unnamed: 22,Unnamed: 23,Unnamed: 24,AttemptID,Unnamed: 27
count,202.000000,202.000000,0.0,202.000000,202.000000,202.000000,86.000000,86.000000,86.000000,0.0,0.0,0.0,18.000000,0.0
mean,3.069307,3.069307,NaN,1.435644,21.014851,81.980198,7.406977,82.383721,0.337209,NaN,NaN,NaN,1.833333,NaN
std,0.959451,0.959451,NaN,0.704157,34.086002,16.452543,4.459960,13.735457,0.475530,NaN,NaN,NaN,1.098127,NaN
min,2.000000,2.000000,NaN,1.000000,1.000000,0.000000,1.000000,30.000000,0.000000,NaN,NaN,NaN,1.000000,NaN
25%,2.000000,2.000000,NaN,1.000000,3.000000,80.000000,3.000000,80.000000,0.000000,NaN,NaN,NaN,1.000000,NaN
50%,3.000000,3.000000,NaN,1.000000,8.000000,90.000000,6.500000,90.000000,0.000000,NaN,NaN,NaN,1.000000,NaN
75%,4.000000,4.000000,NaN,2.000000,11.000000,90.000000,12.000000,90.000000,1.000000,NaN,NaN,NaN,2.750000,NaN
max,4.000000,4.000000,NaN,3.000000,99.000000,100.000000,13.000000,100.000000,1.000000,NaN,NaN,NaN,4.000000,NaN


## User Confirmation Rates

### Attempt-level confirmation rate

In [15]:
confirmed_attempts = (df['IsUserConfirmed'] == True).sum()
total = df['IsUserConfirmed'].count()
attempt_conf_rate = confirmed_attempts / total * 100
attempt_conf_rate

33.16831683168317

Users who make many repeated attempts can skew the metric.
It doesn’t tell you if the same question was eventually confirmed later.
It mixes model performance with user persistence.

### Question-level confirmation rate

In [28]:
question_level = (
    df.groupby(['SurveySessionID', 'QuestionID'])['IsUserConfirmed']
      .max()  # True if any attempt was confirmed
)

question_conf_rate = (question_level == True).sum() / question_level.count() * 100
question_conf_rate

80.48780487804879

In [26]:
question_level.describe()

count       82
unique       2
top       True
freq        66
Name: IsUserConfirmed, dtype: object

### User-level confirmation rate

In [36]:
user_level = (
    df.groupby('SurveySessionID')['IsUserConfirmed']
      .max()  # True if any attempt for any question was confirmed
)

user_conf_rate = (user_level == True).sum() / user_level.count() * 100
print(f"User-level confirmation rate: {user_conf_rate:.2f}%")

User-level confirmation rate: 84.21%


### First-attempt accuracy

## AI Model Accuracy